## Run some HLS composites for wherever the AOI indicates
https://lpdaac.usgs.gov/resources/e-learning/getting-started-cloud-native-hls-data-python/

In [1]:
#!pip install -U -r /projects/code/icesat2_boreal/dps/requirements_main.txt
#!pip install pystac_client
from platform import python_version
import sys
import os

from pyproj import CRS, Transformer
import geopandas as gpd

import pandas as pd

#sys.path.append('/projects/icesat2_boreal/lib')
sys.path.append('/projects/code/icesat2_boreal/lib')
import ExtractUtils
import maplib_folium
import glob
import datetime

/tmp/ipykernel_5828/2370594346.py:8: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
python_version()

'3.10.8'

In [3]:
!pip install -U plotnine

  Using cached numpy-1.24.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2


In [4]:
INDEX_FN = '/projects/my-public-bucket/hls_book_chapter/aoi-book-chapter-usa-conus-square.gpkg'

aoi_gdf = gpd.read_file(INDEX_FN)
aoi_gdf.head()

,geometry
0,"POLYGON ((434629.936 181926.190, 934629.936 18..."


In [5]:
import shapely
def create_tiles_from_aoi(gdf, dims=3000):
    in_crs = gdf.crs
    try:
        
        minx, miny, maxx, maxy = gdf.total_bounds

        #create a fishnet: https://spatial-dev.guru/2022/05/22/create-fishnet-grid-using-geopandas-and-shapely/
        gdf_tile = gpd.GeoDataFrame(index=[0],geometry=[shapely.geometry.Polygon([[minx,miny],[minx,maxy],[maxx,maxy],[maxx, miny]])],crs=in_crs)
    
        # Get the extent of the shapefile
        minX, minY, maxX, maxY = gdf_tile.total_bounds

        # Create a fishnet
        x, y = (minX, minY)
        geom_array = []

        # Polygon Size
        square_size = dims*30
        while y <= maxY:
            while x <= maxX:
                geom = shapely.geometry.Polygon([(x,y), (x, y+square_size), (x+square_size, y+square_size), (x+square_size, y), (x, y)])
                geom_array.append(geom)
                x += square_size
            x = minX
            y += square_size

        # Convert fishnet to gdf
        fishnet = gpd.GeoDataFrame(geom_array, columns=['geometry']).set_crs(in_crs)

        fishnet.index.names = ['tile_num']
        fishnet.reset_index(inplace=True)
        fishnet['tile_num'] += 1

    except Exception as e:
        raise e
    
    return fishnet

aoi_tiles_gdf = create_tiles_from_aoi(aoi_gdf, dims=3000)
#.boundary.explore()
aoi_tiles_gdf.info()
NEW_INDEX_FN = INDEX_FN.replace('.gpkg','_with_id.gpkg')
aoi_tiles_gdf.to_file(NEW_INDEX_FN, driver='GPKG')
NEW_INDEX_FN

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   tile_num  36 non-null     int64   
 1   geometry  36 non-null     geometry
dtypes: geometry(1), int64(1)
memory usage: 704.0 bytes


'/projects/my-public-bucket/hls_book_chapter/aoi-book-chapter-usa-conus-square_with_id.gpkg'

In [8]:
aoi_tiles_gdf.explore()

In [6]:
# z = gpd.read_file('/projects/shared-buckets/nathanmthomas/boreal_tiles_v003.gpkg')
# z.info()

In [7]:
import fiona
fiona.listlayers(NEW_INDEX_FN)

['aoi-book-chapter-usa-conus-square_with_id']

In [7]:
MS_DATA_TYPE = 'HLS'#'LS8' # 'HLS'
if MS_DATA_TYPE == 'LS8':
    SAT_API = 'https://landsatlook.usgs.gov/sat-api'
else:
    SAT_API = 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD'   

In [8]:
#list(range(1,36+1))

In [9]:
import fiona
from CovariateUtils import *

In [10]:
nowtime = pd.Timestamp.now().strftime('%Y%m%d_%H')

In [40]:
DPS_INPUT_TILE_NUM_LIST = aoi_tiles_gdf.tile_num.to_list()

#DPS_INPUT_TILE_NUM_LIST = [3245]

len(DPS_INPUT_TILE_NUM_LIST)

36

In [48]:
MAX_CLOUDS = 50
SEASON_START = '06-01'
SEASON_STOP = '06-30'
YEAR_START = '2019'
YEAR_STOP = '2019'

RUN_NAME_IDENTIFIER = f'mc{MAX_CLOUDS}_{SEASON_START}_{SEASON_STOP}_{YEAR_START}_{YEAR_STOP}'
RUN_NAME_IDENTIFIER

'mc50_06-01_06-30_2019_2019'

In [49]:
# MAAP algorithm version name
MAAP_VERSION = "HLS_stack_2023_v1"

In [51]:
from maap.maap import MAAP
maap = MAAP()
maap._MAAP_HOST

'api.maap-project.org'

In [ ]:
%%time
submit_results_df_list = []
len_input_list = len(DPS_INPUT_TILE_NUM_LIST)
print(f"# of input tiles for DPS: {len_input_list}")

for YEAR in [2019,2020,2021,2022]:
    
    YEAR_START = str(YEAR)
    YEAR_STOP = str(YEAR)
    
    for MONTH in [6,7,8]:

        if MONTH == 6 or MONTH == 9:
            DAY_MAX = 30
        else:
            DAY_MAX = 31

        SEASON_START = f'0{MONTH}-01'
        SEASON_STOP = f'0{MONTH}-{DAY_MAX}'
        MAX_CLOUDS = 50
        
        RUN_NAME_IDENTIFIER = f'mc{MAX_CLOUDS}_{SEASON_START}_{SEASON_STOP}_{YEAR}_{YEAR}'

        for i, INPUT_TILE_NUM in enumerate(DPS_INPUT_TILE_NUM_LIST):

            DPS_num = i+1
            IDENTIFIER = RUN_NAME_IDENTIFIER 
            ALGO_ID = "do_HLS_stack_3-1-2"
            USER = 'montesano'
            WORKER_TYPE = 'maap-dps-worker-32gb'

            in_param_dict = {
                                 #'in_tile_fn': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/boreal_tiles_v003.gpkg',
                'in_tile_fn': 'https://maap-ops-workspace.s3.amazonaws.com/shared/montesano/hls_book_chapter/aoi-book-chapter-usa-conus-square_with_id.gpkg',
                                 'in_tile_num': INPUT_TILE_NUM,
                                 #'in_tile_layer': 'boreal_tiles_v003',
                 'in_tile_layer': 'aoi-book-chapter-usa-conus-square_with_id',
                                 'sat_api': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD',
                                #'sat_api': 'https://landsatlook.usgs.gov/sat-api',
                                 'tile_buffer_m': 0,
                                 'start_year': YEAR_START,
                                 'end_year': YEAR_STOP,
                                 'start_month_day': SEASON_START,
                                 'end_month_day': SEASON_STOP,
                                 'max_cloud': MAX_CLOUDS,
                                 'composite_type': 'HLS',
                                 'shape': 3000,
                                 'hls_product': 'H30'
                }

            submit_result = maap.submitJob(
                                            identifier=IDENTIFIER,
                                            algo_id=ALGO_ID,
                                            version=MAAP_VERSION, # "HLS_stack_2023_v1"
                                            username=USER,
                                            queue=WORKER_TYPE,
                                            # Args that match yaml
                                            **in_param_dict
                )

            # Build a dataframe of submission details
            submit_result_df = pd.DataFrame( 
                {
                        'dps_num':[DPS_num],
                        'tile_num':[INPUT_TILE_NUM],
                        'submit_time':[datetime.datetime.now()],
                        'dbs_job_hour': [datetime.datetime.now().hour],
                        'algo_id': [ALGO_ID],
                        'user': [USER],
                        'worker_type': [WORKER_TYPE],
                        'job_id': [submit_result.id],
                        'submit_status': [submit_result.retrieve_status()],
                } 
            )
            
            submit_result_df['run_name'] = RUN_NAME_IDENTIFIER
            
            # Append to a list of data frames of submission results
            submit_results_df_list.append(submit_result_df)

            if DPS_num in [1, 5, 10, 100, 500, 1000, 1500, 2000, 3000, 5000, 7000, 9000, 11000, 13000, 15000, 17000, 19000, 21000, 24000, len_input_list]:
                print(f"DPS run #: {DPS_num}\t| tile num: {INPUT_TILE_NUM}\t| submit status: {submit_result.retrieve_status()}\t| job id: {submit_result.id}") 

        # Build a final submission results df and save
        submit_results_df = pd.concat(submit_results_df_list)
        submit_results_df['run_name'] = RUN_NAME_IDENTIFIER
        nowtime = pd.Timestamp.now().strftime('%Y%m%d%H%M')
        print(f"Current time:\t{nowtime}")
        submit_results_df.to_csv(f'/projects/my-public-bucket/dps_submission_results/DPS_{IDENTIFIER}_submission_results_{len_input_list}_{nowtime}.csv')
        #print(submit_results_df)

After almost any DPS job, you have to assess what succeeded and failed. This involves:
1. building a table of job status based on job ids captured in the job_results_df from the DPS run chunk (this takes 40 mins for ~47k jobs) --> this tells you how many jobs failed
2. merging the job status table with the job results df --> this tells you which specific granules (or tile nums) failed
3. building another input list of granules for a follow-up DPS
## Assess DPS results
Build a table of job status based on job id - how many jobs failed?

#### Update the dict of DPS runs to monitor success and fails

In [4]:
LIST_SUBMISSIONS = sorted(glob.glob(f'/projects/my-public-bucket/dps_submission_results/DPS_*_submission_results_36*.csv'),key=ExtractUtils.func, reverse=True)
LIST_SUBMISSIONS

['/projects/my-public-bucket/dps_submission_results/DPS_mc50_08-01_08-31_2022_2022_submission_results_36_202307200128.csv',
 '/projects/my-public-bucket/dps_submission_results/DPS_mc50_07-01_07-31_2022_2022_submission_results_36_202307200125.csv',
 '/projects/my-public-bucket/dps_submission_results/DPS_mc50_06-01_06-30_2022_2022_submission_results_36_202307200122.csv',
 '/projects/my-public-bucket/dps_submission_results/DPS_mc50_08-01_08-31_2021_2021_submission_results_36_202307200118.csv',
 '/projects/my-public-bucket/dps_submission_results/DPS_mc50_07-01_07-31_2021_2021_submission_results_36_202307200115.csv',
 '/projects/my-public-bucket/dps_submission_results/DPS_mc50_06-01_06-30_2021_2021_submission_results_36_202307200112.csv',
 '/projects/my-public-bucket/dps_submission_results/DPS_mc50_08-01_08-31_2020_2020_submission_results_36_202307200109.csv',
 '/projects/my-public-bucket/dps_submission_results/DPS_mc50_07-01_07-31_2020_2020_submission_results_36_202307200106.csv',
 '/proje

In [5]:
%%time
#for DPS_DATETIME in [nowtime]:
fails_list = []
success_list = []
#for DICT_RUN_NAME, DPS_DATETIME in sorted(DICT_RUN_NAME_TIME.items(), reverse=True):
for fn in LIST_SUBMISSIONS[0:1]:
    #if DPS_DATETIME in fn and not 'job_status' in fn:
    DPS_alg_id = os.path.basename(fn.split('_submission_results_')[0].replace('DPS_',''))
    thentime = fn.split('_')[-1].replace('.csv','')
    print(f'DPS alg:\t\t{DPS_alg_id}')
    #print(f'DPS run name:\t\t{DICT_RUN_NAME}')
    print(f'DPS launch time:\t{thentime}')
    z = ExtractUtils.BUILD_TABLE_JOBSTATUS(pd.read_csv(fn))
    ## Save job status table
    #z.to_csv(f'/projects/my-public-bucket/dps_submission_results/DPS_{IDENTIFIER}_submission_results_job_status_{len(z)}_{thentime}.csv')

    # Get current fails df and append to list
    #z['run_type'] = DICT_RUN_NAME
    fails_list.append(z[ (z['status'] != 'Succeeded') & (z['status'] != 'Running') ] )
    success_list.append(z[ (z['status'] == 'Succeeded') ] )
            
df_all_fails = pd.concat(fails_list)
df_all_success = pd.concat(success_list)

DPS alg:		mc50_08-01_08-31_2022_2022
DPS launch time:	202307200128
Count total jobs:	432
Count pending jobs:	131
Count running jobs:	3
Count succeeded jobs:	291
Count failed jobs:	7
% of failed jobs:	2.35

CPU times: user 7.67 s, sys: 349 ms, total: 8.02 s
Wall time: 13.2 s


In [8]:
!pip install cogeo_mosaic
import cogeo_mosaic

  Using cached cogeo_mosaic-6.2.0-py3-none-any.whl (38 kB)
  Using cached supermorecado-0.1.1-py3-none-any.whl (14 kB)
  Using cached rio_tiler-5.0.3-py3-none-any.whl (210 kB)
  Using cached morecantile-4.3.0-py3-none-any.whl (36 kB)
  Attempting uninstall: morecantile
    Found existing installation: morecantile 3.4.0
    Uninstalling morecantile-3.4.0:
      Successfully uninstalled morecantile-3.4.0
  Attempting uninstall: rio-tiler
    Found existing installation: rio-tiler 4.1.11
    Uninstalling rio-tiler-4.1.11:
      Successfully uninstalled rio-tiler-4.1.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
rio-cogeo 3.5.1 requires morecantile<4.0,>=3.1, but you have morecantile 4.3.0 which is incompatible.


In [9]:
cogeo_mosaic.__version__

'6.2.0'

In [10]:
MS_COMP_NAME = 'HLS_H30_JC_AOI' #'c2020' #'c2020_fix_additional_irregulars'
MAAP_VERSION = 'HLS_stack_2023_v1' #"HLS_stack_2022_v2"

DPS_MONTH_LIST = '07'
DPS_DAY_MIN = 1
OUTDIR = f'/projects/my-public-bucket/DPS_tile_lists/HLS/{MS_COMP_NAME}/{MAAP_VERSION}'
!mkdir -p $OUTDIR
!time python /projects/code/icesat2_boreal/lib/build_tindex_master.py --RETURN_DUPS --user 'montesano' --maap_version $MAAP_VERSION -alg_name 'do_HLS_stack_3-1-2' -t 'HLS' -y 2023 --dps_month_list $DPS_MONTH_LIST -d_min $DPS_DAY_MIN --outdir $OUTDIR
#!python /projects/Developer/icesat2_boreal/lib/build_tindex_master.py -h

NASA MAAP

Building a list of tiles:
MAAP version:		HLS_stack_2023_v1
Type:		HLS
Year:		2023
Month:		['07']
Days:		1-31

Output dir:  /projects/my-public-bucket/DPS_tile_lists/HLS/HLS_H30_JC_AOI/HLS_stack_2023_v1
                                             s3_path  ...                             file
0  s3://maap-ops-workspace/montesano/dps_output/d...  ...  HLS_3_06-01_06-30_2019_2019.tif
1  s3://maap-ops-workspace/montesano/dps_output/d...  ...  HLS_1_06-01_06-30_2019_2019.tif
2  s3://maap-ops-workspace/montesano/dps_output/d...  ...  HLS_2_06-01_06-30_2019_2019.tif
3  s3://maap-ops-workspace/montesano/dps_output/d...  ...  HLS_4_06-01_06-30_2019_2019.tif
4  s3://maap-ops-workspace/montesano/dps_output/d...  ...  HLS_5_06-01_06-30_2019_2019.tif

[5 rows x 3 columns]
/projects/code/icesat2_boreal/lib/build_tindex_master.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c